In [1]:
import nltk
import pandas as pd
import torch

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /users/eleves-b/2021/guilherme.vieira-
[nltk_data]     manhaes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
validation_df = pd.read_csv('../data/validation.csv')
train_df = pd.read_csv('../data/train.csv')

In [4]:
from nltk import word_tokenize, sent_tokenize

In [5]:
word_tokenize("Bonjour, je m`apelle Guile.", language="french")

['Bonjour', ',', 'je', 'm', '`', 'apelle', 'Guile', '.']

In [6]:
train_df['text_sents'] = train_df['text'].apply(lambda x: sent_tokenize(x, language='french'))

In [33]:
validation_df['text_sents'] = validation_df['text'].apply(lambda x: sent_tokenize(x, language='french'))

In [7]:
train_df.head()

,text,titles,text_sents
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...,[Thierry Mariani sur la liste du Rassemblement...
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...,[C'est désormais officiel : Alain Juppé n'est ...
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a...",[La mesure est décriée par les avocats et les ...
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt...",[Dans une interview accordée au Figaro mercred...
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...,[Le préjudice est estimé à 2 millions d'euros....


In [8]:
device="cuda"

In [9]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-base").to(device)

2024-03-11 18:16:15.548940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:16:15.548965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:16:15.549571: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 18:16:15.553336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 18:16:16.539768: W tensorflow/compiler/tf2

In [10]:
sentences = ["Un avion est en train de décoller.",
          "Un homme joue d'une grande flûte.",
          "Un homme étale du fromage râpé sur une pizza.",
          "Une personne jette un chat au plafond.",
          "Une personne est en train de plier un morceau de papier.",
          ]

embeddings = model.encode(sentences)

In [11]:
PAD = torch.tensor(model.encode(["<pad>"]), device=device)

In [12]:
_, embedding_dim = embeddings.shape

In [13]:
max_sents = 30


In [14]:
embeddings = train_df['text_sents'][:2].apply(lambda x: torch.tensor(model.encode(x), device=device))

In [15]:
embeddings[0].device

device(type='cuda', index=0)

In [16]:
tuple(embeddings)

(tensor([[-0.0040, -0.1797,  0.0429,  ...,  0.0393,  0.1627,  0.0575],
         [-0.0007, -0.1638, -0.1531,  ...,  0.0176,  0.0171,  0.0803],
         [-0.0020, -0.2872,  0.0242,  ...,  0.0603, -0.0371,  0.0529],
         ...,
         [-0.0149,  0.0250,  0.0175,  ...,  0.0523, -0.0266, -0.0365],
         [ 0.0403, -0.0288, -0.0806,  ...,  0.1663, -0.0497,  0.1327],
         [-0.0327, -0.0042, -0.0528,  ...,  0.0492,  0.1555,  0.0305]],
        device='cuda:0'),
 tensor([[-0.1212, -0.3656,  0.0027,  ...,  0.0868,  0.0333,  0.0712],
         [-0.0128, -0.3151, -0.0098,  ..., -0.0376,  0.1096,  0.0345],
         [ 0.0843, -0.2349,  0.1499,  ..., -0.0067,  0.0260,  0.1409],
         ...,
         [-0.0828, -0.1743, -0.0120,  ...,  0.0573,  0.0283,  0.0830],
         [-0.0232, -0.0006,  0.0304,  ...,  0.1469, -0.0011,  0.1170],
         [ 0.0497, -0.1555,  0.0136,  ..., -0.0176,  0.0851,  0.1304]],
        device='cuda:0'))

In [17]:
PAD.repeat(max_sents, 1).shape

torch.Size([30, 768])

In [18]:
def pad(embedding):
    n, _ = embedding.shape
    t = PAD.repeat(max_sents, 1)
    if n < max_sents:
        t[:n, :] = embedding
    else:
        t = embedding[:max_sents]
    return t

In [19]:
tuple(pad(emb) for emb in embeddings)

(tensor([[-0.0040, -0.1797,  0.0429,  ...,  0.0393,  0.1627,  0.0575],
         [-0.0007, -0.1638, -0.1531,  ...,  0.0176,  0.0171,  0.0803],
         [-0.0020, -0.2872,  0.0242,  ...,  0.0603, -0.0371,  0.0529],
         ...,
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187],
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187],
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187]],
        device='cuda:0'),
 tensor([[-0.1212, -0.3656,  0.0027,  ...,  0.0868,  0.0333,  0.0712],
         [-0.0128, -0.3151, -0.0098,  ..., -0.0376,  0.1096,  0.0345],
         [ 0.0843, -0.2349,  0.1499,  ..., -0.0067,  0.0260,  0.1409],
         ...,
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187],
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187],
         [-0.1313,  0.0690,  0.0688,  ...,  0.1028,  0.1254,  0.3187]],
        device='cuda:0'))

In [20]:
train  = torch.stack(tuple(pad(emb) for emb in embeddings))

In [21]:
train.shape

torch.Size([2, 30, 768])

In [22]:
train.device

device(type='cuda', index=0)

In [23]:
model.tokenizer.vocab_size

32005

In [24]:
model.tokenizer("je")

{'input_ids': [5, 50, 6], 'attention_mask': [1, 1, 1]}

In [25]:
model.tokenizer.batch_decode([[1, 3, 2], [5, 71, 6]])

['<pad><unk></s>NOTUSED', '<s> L</s>']

In [26]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, max_out_len, vocab_size, embed_size):
        super(MyModel, self).__init__()
        self.max_out_len = max_out_len
        self.vocab_size = vocab_size
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        # Assuming x is of shape (batch_size, max_len, embed_size)
        batch_size, max_len, _ = x.shape

        x = self.fc(x)

        if max_len < self.max_out_len:
            pad_size = self.max_out_len - max_len
            pad =  torch.zeros(batch_size, pad_size, self.vocab_size, device=x.device)
            x = torch.cat([x, pad], dim=1)
        elif max_len > self.max_out_len:
            x = x[:, :self.max_out_len, :]
        return x

In [27]:
mm  = MyModel(50, model.tokenizer.vocab_size, embedding_dim).to(device)


In [28]:
mm(train)

tensor([[[-0.0257,  0.1379, -0.0053,  ..., -0.0098, -0.1397,  0.0512],
         [ 0.1126, -0.0018, -0.0338,  ...,  0.1652, -0.0043,  0.0861],
         [-0.0268,  0.0823, -0.0430,  ...,  0.0397, -0.1553,  0.0462],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0442,  0.0461,  0.0152,  ...,  0.0500, -0.0554,  0.0354],
         [ 0.0585,  0.0606, -0.0103,  ...,  0.0215, -0.1264,  0.0855],
         [ 0.0195,  0.0480, -0.0528,  ...,  0.0241, -0.0142, -0.0079],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [29]:
model.tokenizer.pad_token_id

1

In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe, features, target):
        self.data = dataframe
        self.features = features
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = torch.tensor(model.encode(self.data.iloc[idx][self.features]))
        y = model.tokenizer(self.data.iloc[idx][self.target], max_length=50, padding='max_length', truncation=True)['input_ids']
        return (
            pad(x).to(device), 
            torch.stack(tuple(torch.tensor(t, dtype=torch.int64, device=device) for t in y))
        )
    
dataset = CustomDataset(train_df, features='text_sents', target='titles')

batch_size = 64
shuffle = True
num_workers = 2

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mm.parameters(), lr=0.001)
num_epochs = 2
mm.to(device)

for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        # Forward pass
        outputs = mm(inputs)  # Shape of outputs: (batch_size, max_out_len, vocab_size)
        outputs = outputs.view(-1, outputs.shape[-1])  # Reshape for cross-entropy loss
        targets = targets.view(-1)  # Reshape targets to match output
        # print(inputs.shape, targets.shape, outputs.shape)
        # print(inputs.device, targets.device, outputs.device)

        # Compute loss
        loss = criterion(outputs, targets)
        # print(loss)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/2, Loss: 8.650711059570312
Epoch 2/2, Loss: 8.521456718444824


In [35]:
dataset = CustomDataset(validation_df, features='text_sents', target='titles')


data_loader_val = DataLoader(dataset)

In [41]:
for inputs, targets in data_loader_val:
    out = mm(inputs)
    out = torch.argmax(out,dim=-1)
    dec = model.tokenizer.batch_decode(out)[0]
    dec = (
        dec
        .replace("<s>", "")
        .replace("<pad>", "")
        .replace("NOTUSED", "")
    )
    print(dec)

 de de
' de
 de



 de de''
'' de de de de de
 Un' Un Un


 de de, de de
 de de de
 de

 de de de de

' de
 de de'
 de, de



 de de de de'
 de' de de
e
 de'

 de de, de
 de','
'

 de, de, de la de',

,

',

 de', de' de

 de'' de
' de' Le'
 de de,
,,
'


 de de
 Un
 de'
' La de de de de de

'
 de
de de' de
'' de de
 de' de'

AGE

 de
',
'
 de
' de de de de' de

de de

 Le Le de de
'
 de'
'' de'
 de
 de de
'
de
'', de

ministre de de de,,,' confi,
 Un'
 de de de de de de
 de de'
,

 de
 Un'' de' homme Le'
 de de L de
 de
'
 de de de de,
 de de
' de de

 de

 de
de de de' de'

 de'
 ministre de
'' de
de, de de de de' de
 de'
 de de de de''''
'' de de,
 de

 de de de de de'
 de' Le
 a de

 Un Un

 de'
 de de

 de de
'
 de ministre
'''
 L' Les de, Shar,'

 de
de de de de de de
 de
'
 La de de, de' de de de de
 de' de'' de'
,
 de'
,'

' La


 de de de
 de
 de' de de'

 de de de de de' de de,
' de
''
 de de de de
'
' de de
'OLOND Un Les
''
 de, de de de de
,, de
' Météo
de Un de


de de

,'' de
'
 de de de
'''
 Le Le
 de
 Un
 de de
 de

, de de de'

KeyboardInterrupt: 